In [87]:
import spacy 
import numpy as np
import emb_spacy
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional, GRU
from keras.metrics import Accuracy, Precision, Recall, F1Score, MeanSquaredError




from emb_spacy import get_embedding
from input import count_examples_and_max_length, pad_sentences_from_file
import label

### Lecture des données 

In [88]:
# Ouverture en lecture des données
with open("train_corpus", "r", encoding="utf-8") as file:
    data = file.readlines()

# Compter les exemples et trouver la taille maximale
num_examples, MAX_SEQ_SIZE = count_examples_and_max_length(data)

# Affichage des résultats
print("Nombre d'exemples :", num_examples) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE)

vec_word, sortie = pad_sentences_from_file("train_corpus", MAX_SEQ_SIZE)
print("vecteur : ", vec_word.shape)
# vec_word.shape : (3945, 41)
# sortie.shape : (3945, 41)

Nombre d'exemples : 3945
Taille maximale de la phrase : 41
vecteur :  (3945, 41)


### Création des vecteurs d'entrée et de sortie

In [89]:
# Création du vecteur d'entrée 
entree = np.zeros((num_examples, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word):
    for j, word in enumerate(sentence):
        if word:
            entree[i, j] = get_embedding(word)
        else:
            entree[i, j] = np.zeros(300)

print("entree shape : ", entree.shape)
print("sortie shape : ", sortie.shape)

entree shape :  (3945, 41, 300)
sortie shape :  (3945, 41)


In [90]:
labels = label.extract_label("atis.train")
label_one_hot_dict = {label_: label.get_vector_from_label(label_) for label_ in labels} # création d'un dictionnaire associant chaque label à son vecteur one hot 

nbLabels = len(labels)
embedding_size = len(entree[0][0])

tailleDictionnaire = emb_spacy.get_size_dict()  

In [91]:
# print(sortie[0])
sortie_one_hot = np.zeros((sortie.shape[0], sortie.shape[1], len(label_one_hot_dict['O'])), dtype=int)
zero_vec = np.zeros(len(label_one_hot_dict['O']), dtype=int)
for i in range(sortie.shape[0]):
    for j in range(sortie.shape[1]):
        label = sortie[i, j]
        if label == '0':
            sortie_one_hot[i, j] = zero_vec
        else:
            sortie_one_hot[i, j] = label_one_hot_dict[label]

### Modèle LSTM

#### Configuration du modèle

In [92]:

config = {
    'hidden_size': 128, # Taille de la couche cachée du RNN
    'dropout_rate': 0.5,  # Taux de dropout
    'nb_labels': nbLabels
}  

# Définir l'entrée du modèle
# pas besoin de mettre le nb d'ex car il les fait passer un par un 
input_layer = Input(shape=(MAX_SEQ_SIZE, embedding_size), dtype='float32')

# Ajouter une couche LSTM bidirectionnelle
# X = Bidirectional(LSTM(units=config['hidden_size'], return_sequences=True))(input_layer)
X = LSTM(units=config['hidden_size'], return_sequences=True)(input_layer)
X = Dropout(config['dropout_rate'])(X)
# X = LSTM(units = 128)(X)
# X = Dropout(config['dropout_rate'])(X)

X = Dense(units=config['nb_labels'])(X)
X = Activation(activation='softmax')(X)
print('config ok')

# Création du model
model = Model(inputs=input_layer, outputs=X)
print("create ok")

# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
# NB : j'ai enlevé sparse car j'ai fait un vecteur one hot en sortie et sparse c'est pour quand c'est pas des one hot
print("compil ok")
print(model.summary())

config ok
create ok
compil ok


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 41, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 41, 128)        │       219,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 41, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 41, 81)         │        10,449 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 41, 81)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 230,097 (898.82 KB)

 Trainable params: 230,097 (898.82 KB)

 Non-trainable params: 0 (0.00 B)

None


### Entraînement du modèle LSTM
- x : entree de la forme nb_example x MAX_SEQ_SIZE x embedding_size
- y : sortie de la forme nb_example x MAX_SEQ_SIZE
- batch_size : nb d'échantillon à utiliser à chaque itération lors de l'entrainement
    - un batch_size + gd = accélère l'entrainement mais besoin de + de mémoire GPU
    - plus petit = ralentit l'entrainement mais meilleure convergence du modèle
- epochs : nb d'itération sur l'ens des données d'entrainement 
- validation_split : spécifie le fraction des données à utiliser comme données de validation

In [93]:
model.fit(x=entree, y=sortie_one_hot, batch_size=128, epochs=10, validation_split=0.2)
print('training ok')

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.1538 - loss: 0.6819 - precision_7: 0.8401 - recall_7: 0.2003 - val_accuracy: 0.2111 - val_loss: 0.2086 - val_precision_7: 0.9401 - val_recall_7: 0.7784
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.2982 - loss: 0.2005 - precision_7: 0.9531 - recall_7: 0.7884 - val_accuracy: 0.9591 - val_loss: 0.1522 - val_precision_7: 0.9556 - val_recall_7: 0.8377
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.3537 - loss: 0.1395 - precision_7: 0.9700 - recall_7: 0.8431 - val_accuracy: 0.9669 - val_loss: 0.1305 - val_precision_7: 0.9610 - val_recall_7: 0.8546
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.3801 - loss: 0.1122 - precision_7: 0.9743 - recall_7: 0.8702 - val_accuracy: 0.9561 - val_loss: 0.1156 - val_precision_7: 0.9624 - val_recall_7: 0.8641
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.3653 - loss: 0.0989 - precision_7: 0.9762 - recall_7: 0.8834 - val

### Modèle GRU

#### Configuration du modèle

In [94]:

config = {
    'hidden_size': 128, # Taille de la couche cachée du RNN
    'dropout_rate': 0.5,  # Taux de dropout
    'nb_labels': nbLabels
}  

# Définir l'entrée du modèle
# pas besoin de mettre le nb d'ex car il les fait passer un par un 
input_layer = Input(shape=(MAX_SEQ_SIZE, embedding_size), dtype='float32')

# Ajouter une couche LSTM bidirectionnelle
# X = Bidirectional(LSTM(units=config['hidden_size'], return_sequences=True))(input_layer)
X = GRU(units=config['hidden_size'], return_sequences=True)(input_layer)
X = Dropout(config['dropout_rate'])(X)
# X = LSTM(units = 128)(X)
# X = Dropout(config['dropout_rate'])(X)

X = Dense(units=config['nb_labels'])(X)
X = Activation(activation='softmax')(X)
print('config ok')

# Création du model
model_GRU = Model(inputs=input_layer, outputs=X)
print("create ok")

# Compilation du modèle
model_GRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
# NB : j'ai enlevé sparse car j'ai fait un vecteur one hot en sortie et sparse c'est pour quand c'est pas des one hot
print("compil ok")
print(model_GRU.summary())

config ok
create ok
compil ok


Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 41, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 41, 128)        │       165,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 41, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 41, 81)         │        10,449 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 41, 81)         │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 175,569 (685.82 KB)

 Trainable params: 175,569 (685.82 KB)

 Non-trainable params: 0 (0.00 B)

None


#### Entrainement du modèle

In [95]:
model_GRU.fit(x=entree, y=sortie_one_hot, batch_size=128, epochs=10, validation_split=0.2)
print('training ok')

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 92ms/step - accuracy: 0.1628 - loss: 0.6850 - precision_8: 0.8249 - recall_8: 0.2774 - val_accuracy: 0.8201 - val_loss: 0.2199 - val_precision_8: 0.9474 - val_recall_8: 0.7521
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.3934 - loss: 0.2290 - precision_8: 0.9458 - recall_8: 0.7563 - val_accuracy: 0.9329 - val_loss: 0.1770 - val_precision_8: 0.9586 - val_recall_8: 0.8124
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.3932 - loss: 0.1856 - precision_8: 0.9573 - recall_8: 0.7933 - val_accuracy: 0.9547 - val_loss: 0.1561 - val_precision_8: 0.9644 - val_recall_8: 0.8339
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.3978 - loss: 0.1600 - precision_8: 0.9619 - recall_8: 0.8220 - val_accuracy: 0.9596 - val_loss: 0.1419 - val_precision_8: 0.9668 - val_recall_8: 0.8429
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.4020 - loss: 0.1442 - precision_8: 0.9655 - recall_8: 0.8402 - val_

### Prétraitement des données de test

In [96]:
with open("test_corpus", "r", encoding="utf-8") as file:
    data_test = file.readlines()
    
num_examples_test, MAX_SEQ_SIZE_TEST = count_examples_and_max_length(data_test)


vec_word_test, label_test_real = pad_sentences_from_file("test_corpus", MAX_SEQ_SIZE)

print("Nombre d'exemples :", num_examples_test) 
print("Taille maximale de la phrase :", MAX_SEQ_SIZE_TEST)

# Création du vecteur d'entrée 
test_data_input = np.zeros((num_examples_test, MAX_SEQ_SIZE, 300))
for i, sentence in enumerate(vec_word_test):
    for j, word in enumerate(sentence):
        if word:
            test_data_input[i, j] = get_embedding(word)
        else:
            test_data_input[i, j] = np.zeros(300)

Nombre d'exemples : 1033
Taille maximale de la phrase : 32


In [97]:
print(test_data_input.shape)
print(label_test_real.shape)
print(entree.shape)

(1033, 41, 300)
(1033, 41)
(3945, 41, 300)


In [98]:
label_test_real_one_hot = np.zeros((label_test_real.shape[0], label_test_real.shape[1], len(label_one_hot_dict['O'])), dtype=int)
zero_vec = np.zeros(len(label_one_hot_dict['O']), dtype=int)
for i in range(label_test_real.shape[0]):
    for j in range(label_test_real.shape[1]):
        label = label_test_real[i, j]
        if label == '0':
            label_test_real_one_hot[i, j] = zero_vec
        else:
            label_test_real_one_hot[i, j] = label_one_hot_dict[label]

print(label_test_real_one_hot.shape)

(1033, 41, 81)


### Prédictions

#### Avec le modèle LSTM

In [99]:
test_predictions = model.predict(test_data_input)

print(test_predictions.shape)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
(1033, 41, 81)


In [100]:
predicted_labels_index = np.argmax(test_predictions, axis=-1)
# print(predicted_labels_index[10])
predicted_labels = [[labels[idx] for idx in sample] for sample in predicted_labels_index]
predicted_labels = np.array(predicted_labels)
# print(predicted_labels[10])

In [101]:
_ = model.evaluate(test_data_input, label_test_real_one_hot)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9796 - loss: 0.0828 - precision_7: 0.9655 - recall_7: 0.8971


In [102]:
def write_predictions_to_file(file_path, vec_word_test, label_test_real, predicted_labels):
    with open(file_path, 'w', encoding="utf-8") as file:
        for words, real_labels, predicted_labels in zip(vec_word_test, label_test_real, predicted_labels):
            for word, real_label, predicted_label in zip(words, real_labels, predicted_labels):
                if real_label == '0':
                    break
                word_length = max(len(word), 8)
                file.write(f"{word.ljust(word_length)}\t{real_label.ljust(10)}\t{predicted_label}\n")
            file.write("===========================================================\n")

In [103]:
write_predictions_to_file("predictions_LSTM.txt", vec_word_test, label_test_real, predicted_labels)

#### Avec le modèle GRU

In [104]:
test_predictions_GRU = model_GRU.predict(test_data_input)

print(test_predictions_GRU.shape)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step
(1033, 41, 81)


In [105]:
predicted_labels_index_GRU = np.argmax(test_predictions_GRU, axis=-1)
# print(predicted_labels_index[10])
predicted_labels_GRU = [[labels[idx] for idx in sample] for sample in predicted_labels_index_GRU]
predicted_labels_GRU = np.array(predicted_labels_GRU)
# print(predicted_labels[10])

In [106]:
_ = model_GRU.evaluate(test_data_input, label_test_real_one_hot)

33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9776 - loss: 0.1205 - precision_8: 0.9629 - recall_8: 0.8794


In [107]:
write_predictions_to_file("predictions_GRU.txt", vec_word_test, label_test_real, predicted_labels_GRU)